# Problem 4 : Estimation - BLP

In [1]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra,StatsFuns

Random.seed!(8675309);

In [2]:
df = DataFrame(CSV.File("../data/ps1_ex4.csv"));
x = Array(df[!,[:x, :p]]);

Γ = ones(2,2)
Γ[1,2] = 0

In [13]:
# get data
x_t = []
for t in 1:100 
   push!(x_t, Array(df[df[!,:market].==t, [:p, :x]]))
end

s_t = []
for t in 1:100 
    push!(s_t, Array(df[df[!,:market].==t, [:shares]]))
end

z_t = []
for t in 1:100 
    push!(z_t, Array(df[df[!,:market].==t, [:z1, :z2, :z3, :z4, :z5, :z6, :x]]))
end

In [14]:
# simulate individual taste shocks from N(μ,Σ)
draw_sim = function(μ, Σ, N) # return N x L matrix
    # draw shocks
    ν = transpose(rand(MvNormal(μ, Σ), N))
    
    return ν
end

#1 (generic function with 1 method)

# Part 1: BLP

## Inner loop
`get_shares` calculates the shares of each product in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a matrix of size $J \times 2$; and $v$ should be a vector of length $L$.

`delta_contraction` iterates the $\delta_{jt}$ in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a vector of characteristics with length $J$; $s$ should be a vector of observed shares with length $J$; $v$ should be a vector of length $L$. 

`market_iterate` performs the contraction over each $t$ markets, it recoves $\delta_{jt}$, which is a vector of length $J \times T$.

In [55]:
# get shares in a market given some fixed gamma and delta
get_shares = function(δ, Γ, x, v)
    # we want to get share_{jt} using simulated values of $v_i$ (drawn above)
    # shares should be vector of length J
    
    numerator = exp.(δ .+ x * Γ * v')
    denominator = sum(numerator, dims = 1) .+ 1
    shares = sum(numerator ./ denominator, dims = 2) ./ size(v)[1]
    
    return shares
end

# inner loop: contraction to find δ
delta_contraction = function(δ₀, Γ, s, x, v, tol = 1e-12, max_iter = nothing)

    # here δ is a vector of length J
    δ = δ₀
    err = 1000
    n = 0
    maxed_iter = false
    #print(err, maxed_iter)
    
    while (err > tol) && (maxed_iter === false)
        δ_old = δ
        
        # update delta
        δ = δ_old + log.(s) - log.(get_shares(δ_old, Γ, x, v))
        
        # difference 
        err = maximum(abs.(δ - δ_old)) 
        
        # (optional) max iterations block
        n += 1
        if max_iter !== nothing
            maxed_iter = (n == max_iter)
        end
    end
    
    return δ
end

# iterate over each market
market_iterate = function(Γ, s_t, x_t, v, tol = 1e-12, max_iter = nothing)
   
    δ = []
    for t in 1:size(s_t)[1]
        s = s_t[t]
        x = x_t[t]
        δ₀ = ones(size(s)[1])
        push!(δ, delta_contraction(δ₀, Γ, s, x, v, tol, max_iter) ) 
    end
    return δ
end

#31 (generic function with 3 methods)

## Outer loop
`residuals` does IV-GMM using the provided weighting matrix. z_jt should be a matrix of $Z$ excluded and included intruments of size $TJ \times Z$. Returns linear parameters (vector of length $2$) and $\xi_{jt}$ residuals (vector of length $J \times T$)

`gmm_objective` Reads in $TJ$-length vector $x$_jt and $TJ \times Z$ matrix $z$_jt. Calculates sample moments (size of instrument vector, $Z$) and optimal weighting matrix ($Z \times Z$). Returns scalar objective and matrix.

In [83]:
# returns residuals for a given δ, estimates linear parameters given instruments
resid = function(δ_jt, x_jt, z_jt, W = nothing)
    # if weighting matrix not supplied
    if W === nothing
        W = inv(z_jt' * z_jt)
    end
    
    # iv-gmm
    θ₁ = inv(x_jt' * z_jt * W * z_jt' * x_jt) * (x_jt' * z_jt * W * z_jt' * δ_jt)
    ξ_jt = δ_jt - x_jt * θ₁
    
    return ξ_jt, θ₁ 
    
end

# calculates gmm objective for outer loop
function gmm_objective(ξ_jt, z_jt)   
    # empirical moments, weighting matrix
    g = (ξ_jt' * z_jt) / size(ξ_jt)[1] 
    W = inv((z_jt .* ξ_jt.^2 )' * z_jt) * size(ξ_jt)[1]
    
    # gmm objective
    G = g * W * g'
    
    return G, W
end

# performs outer loop, keeps track of weighting matrix
function outer_loop!(θ₂, s_t, x_t, x_jt, z_jt, v, tol = 1e-12, max_iter = nothing,  W = nothing)
    # Pass through guess
    Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    
    # Perform inner loop
    δ = market_iterate(Γ, s_t, x_t, v, tol, max_iter) # what is initial??
    
    # convert to JT x 1 (stacked J x 1 vectors for each t)
    δ_jt = vec(reduce(hcat,δ)) 
    
    # intermediate step
    ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt, W)
    
    # gmm step should this step be optimizing the GMM objective?
    G, W = gmm_objective(ξ_jt, z_jt)
    
    return G, W
end

Optimize(outer_loop)

#60 (generic function with 4 methods)

In [118]:
# Define input objects
input = sort(df,[:market,:choice])

x_t = Matrix(unstack(input, :market, :choice, :x)[:,2:7]);
s_t = Matrix(unstack(input, :market, :choice, :shares)[:,2:7]);
z_jt = Matrix(input[:,[:z1, :z2, :z3, :z4, :z5, :z6, :x]]);
ν = draw_sim([0;0], [1 0;0 1], 100);

x_t

100×6 Matrix{Union{Missing, Float64}}:
 -0.210973   -1.55758    -0.799984   -1.33837    3.31633    2.15453
  0.470776   -0.153134   -2.18476    -0.743471   1.56558    3.14475
 -0.0839182   0.394334   -0.478595   -0.304705   0.276072   0.884968
 -1.11659     1.21433    -0.143263    0.206286   1.83095    2.44176
  0.270833   -0.172235    1.8037      0.193692   1.62421    1.63298
 -0.0095628   0.233098   -0.648853    2.55467    2.11398    1.9928
 -1.0391     -0.653942   -1.56475    -0.861561   0.739625   2.13211
 -1.30205     1.30268     0.511582   -0.666108   0.474729   3.06745
 -1.26007     0.191155   -0.690102    0.656296   0.403315   2.08149
  0.500571    0.104896   -0.0866057  -0.997533   2.53638    0.544624
 -0.0724124  -1.71788     1.15126    -0.524685   2.91933    3.32483
  0.351376    1.18773    -0.741401    0.309983   1.0358     2.72921
 -0.488561   -0.01601     0.665288   -0.999197   3.22408    4.74749
  ⋮                                                         ⋮
  0.939254    

In [48]:
δ₀ = ones(6)
Γ = [1 0 ; 1 1]
ν = draw_sim([0;0], [1 0;0 1], 100);


In [63]:

delt = market_iterate(Γ, s_t, x_t, ν,1e-12,1000);
delt[1]

6×1 Matrix{Float64}:
 -3.618024748837521
 -1.79856074386391
 -3.6748897163554544
 -3.1839842780893397
 -3.3390442808354655
 -4.1117633043894095

In [70]:
x_jt = Array(df[:,[:p, :x]])
z_jt = Matrix(df[:,[:z1, :z2, :z3, :z4, :z5, :z6, :x]]);

In [71]:
δ_jt = vec(reduce(hcat, delt))

600-element Vector{Float64}:
 -3.618024748837521
 -1.79856074386391
 -3.6748897163554544
 -3.1839842780893397
 -3.3390442808354655
 -4.1117633043894095
 -3.8326217285751447
 -2.664766123761004
 -2.811192170564724
 -4.936138103427132
 -3.3838483432643502
 -2.443083653659266
 -2.643432789799815
  ⋮
 -2.556679962297973
 -1.055500992247878
 -3.9281294531086433
 -2.2466523580570694
 -1.7004135865846588
 -1.8650961473346463
 -2.8686185314436985
 -1.9013607323865664
 -6.07384576772032
 -6.200885011053797
 -2.659522140032446
 -4.5654823842996235

In [89]:
x, y = resid(δ_jt, x_jt, z_jt)

([-3.7557669697033567, -2.815738331077342, -4.037894100198456, -4.04089976797594, -0.32823409119227964, -2.047858277408077, -3.5250705870386736, -2.7647286001362383, -4.124467208969504, -5.010726640910749  …  -3.4774667950729565, -1.156103971755376, -0.7584168279413547, -0.48528559374265434, -2.9038864553996167, -2.0584595793512626, -5.718055615827845, -5.757893041206534, -1.539505794205374, -1.4514650462162955], [-0.1568221756254733, -0.6530971610957783])

In [98]:
outer_loop = function(θ₂, s_t, x_t, x_jt, z_jt, v, tol = 1e-12, max_iter = nothing,  W = nothing)
    # Pass through guess
    Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    
    # Perform inner loop
    δ = market_iterate(Γ, s_t, x_t, v, tol, max_iter) # what is initial??
    
    # convert to JT x 1 (stacked J x 1 vectors for each t)
    δ_jt = vec(reduce(hcat,δ)) 
    
    # intermediate step
    ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt, W)
    
    # gmm step
    G, W = gmm_objective(ξ_jt, z_jt)
    
    return G, W
end

#64 (generic function with 4 methods)

In [99]:
g, w = outer_loop([1 1 1], s_t, x_t, x_jt, z_jt, ν)

(0.21896565080926234, [0.085603451180656 -0.034882942297747 … -0.005707878298005312 -0.005566964348651631; -0.03488294229774701 0.08969824619390354 … 0.007811092682178221 -0.005534110983618899; … ; -0.005707878298005316 0.007811092682178222 … 0.12877094892746987 0.002128356730109853; -0.005566964348651642 -0.0055341109836188895 … 0.0021283567301098524 0.09548174494899414])

In [101]:
optimize(θ₂ -> outer_loop!(blah))

0.21896565080926234